In [1]:
from ast import literal_eval
import json
import os
from typing import List

import pandas as pd
from sklearn.model_selection import train_test_split

## Configs

In [2]:
HAM_CSV = "raw/data_v1.csv"
WOHAM_CSV = "raw/data_notagword_v1.csv"

VAL_FROM_TRAIN_RATIO = 0.10

X_COLUMN = "text"
TOKEN_TAG_COLUMN = "tag_by_word"
Y_COLUMN = "tags"
RANDOM_SEED = 42

## Read data

In [3]:
w_ham_converters = {
    "text": literal_eval,
    "tag_by_word": literal_eval
}

w_ham_df = pd.read_csv(HAM_CSV, converters=w_ham_converters)
print(len(w_ham_df))
w_ham_df.head()

4141


,text,tag_by_word,tags
0,"[ประเทศจีน, ชอบ, ปลอม, เอา, ยาง, รถ, ไป, ทำ, ไ...","[I-Influencer, Fb-Refer, Fb-Refer, Fb-Refer, F...",Fake News
1,"[ประกาศ, กฎ, อัยการศึก, ปิด, การ, เข้า, ออก, ท...","[T-Clickbait, T-Clickbait, T-Clickbait, , , , ...",Undefine
2,"[ประโยชน์, มาก, ก็, ใช่, ว่า, จะ, ไม่, มี, โทษ...","[, , , , , , , , , , , M-Convincing, M-Convinc...",Undefine
3,"[มั่นใจ, กระซิบ, หู, หนู, พระพิฆเนศ, ไม่, ติด,...","[T-Clickbait, T-Clickbait, T-Clickbait, T-Clic...",Fake News
4,"[พวก, ที่, เชื่อ, ว่า, คน, ไม่, ป่วย, ไม่, ต้อ...","[, , , , , , , , , , , , , , , , , , , , , , ,...",Undefine


In [4]:
wo_ham_converters = {
    "title": lambda x: literal_eval(x) if x else [],
    "detail": lambda x: literal_eval(x) if x else []
}
wo_ham_df = pd.read_csv(WOHAM_CSV, converters=wo_ham_converters)
print(len(wo_ham_df))


def get_text(_title: List[str], _detail: List[str]) -> List[str]:
    if _detail and not _title:
        return _detail
    elif _title and not _detail:
        return title
    return [*_title, '|', '|', '|', *_detail]

wo_ham_df["text"] = wo_ham_df.apply(lambda x: get_text(x.title, x.detail), axis=1)
wo_ham_df.head()

3050


,title,detail,tags,text
0,[],"[พี่, สาว, คน, ข้าง, บ้าน, เป็น, มะเร็ง, ต่อม,...",Fake News,"[พี่, สาว, คน, ข้าง, บ้าน, เป็น, มะเร็ง, ต่อม,..."
1,[],"[กิน, แทน, ข้าว, หนุ่ม, เวียดนาม, ปวด, ท้อง, 2...",Fake News,"[กิน, แทน, ข้าว, หนุ่ม, เวียดนาม, ปวด, ท้อง, 2..."
2,[],"["", ต้น, ป่าช้า, เหงา, "", สมุนไพร, ไทย, ดัง, ไ...",Fake News,"["", ต้น, ป่าช้า, เหงา, "", สมุนไพร, ไทย, ดัง, ไ..."
3,[],"[ผง, ชู@@, รส, นี่, ก็, สาร, พิษ, ก่อ, เกิด, โ...",Fake News,"[ผง, ชู@@, รส, นี่, ก็, สาร, พิษ, ก่อ, เกิด, โ..."
4,"[รู้, ไว้, มี, ประโยชน์, !, , , ไม่, อยาก, เป็...","[ไม่, อยาก, เป็น, นิ่ว, , , ต้อง, หลีกเลี่ยง, ...",Fake News,"[รู้, ไว้, มี, ประโยชน์, !, , , ไม่, อยาก, เป็..."


In [5]:
print("Total sampling: %d"%(len(w_ham_df) + len(wo_ham_df)))

Total sampling: 7191


## Splitting

In [6]:
X, Y = wo_ham_df[X_COLUMN], wo_ham_df[Y_COLUMN]

In [7]:
x_train, x_val, y_train, y_val = train_test_split(
    X, Y, test_size=VAL_FROM_TRAIN_RATIO,
    stratify=Y, random_state=RANDOM_SEED
)
print(len(x_train), len(y_train))
print(len(x_val), len(y_val))

2745 2745
305 305


In [8]:
x_test, x_tags_test, y_test = w_ham_df[X_COLUMN], w_ham_df[TOKEN_TAG_COLUMN], w_ham_df[Y_COLUMN]
print(len(x_test), len(x_tags_test), len(y_test))

4141 4141 4141


### Remove train/test overlap

In [9]:
train_texts = ["".join(x) for x in X]

nondup_x_test = []
nondup_x_tags_test = []
nondup_y_test = []

n_leak = 0
for x, tags, y in zip(x_test, x_tags_test, y_test):
    if "".join(x) in train_texts:
        n_leak += 1
        print(n_leak, "".join(x))
        continue
    nondup_x_test.append(x)
    nondup_x_tags_test.append(tags)
    nondup_y_test.append(y)

1 ประเทศจีนชอบปลอมเอายางรถไปทำไข่มุกค่ะ
2 มันคือน้ำศักดิ์สิทธิ์ไว้รักษาคนป่วยเป็นมะเร็ง
3 พวกคุณเข้าใจผิดกันไปใหญ่แล้วสิ่งที่พวกคุณเห็นมันไม่ใช่หน้ากากอนามัยแต่มันคือแป้ง..
4 น่ากลัวมากไข่มุกที่มีในน้ำถ้วยมันย่อยยากบางคนไม่ย่อยเลยคาในท้องต้องผ่าตัดออกมาระวังด้วย
5 นั่งฟังเเม่พูดว่าให้กินน้ำอุ่นผสมมะนาวช่วยต้านโควิดอันนี้เรื่องจริงมั้ย?!
6 เบียร์15กระป๋องทำลายเชื้อโควิดได้ครีบ
7 แม่ผัวบอกไข่มุกจีนทำจากยางรถยนต์เลิกกินเลยค่ะ555+
8 น้ำศักดิ์สิทธิ์ครับตักใส่ขวดกลับไปกินเป็นยารักษาโรคได้เลย
9 มันไม่หน้ากากอนามัยมันคือกระดาษเค้าเข้าใจผิด
10 ผลิตภัณฑ์ดีคอนแทคได้รับความไว้วางใจจากลูกค้า
11 ฟื้นฟูการได้ยิน100%โดยไม่มีการผ่าตัดให้คุณใช้ชีวิตโดยไม่ต้องพึ่งเครื่องช่วยฟังไม่ก่อให้เกิดการแพ้และปลอดภัยสำหรับทุกวัยจากธรรมชาติ100%
12 ภาพที่เห็นไม่ใช่หน้ากากอนามัยแต่เป็นผ้าอนามัยตำรวจออสซี่เข้าใจผิดแล้ว
13 มันคือสมุนไพรรักษามะเร็ง
14 มันไม่ย่อยชาไข่มุก
15 มีสมุนไพรรักษามะเร็งมั้ยคะ
16 มีสมุนไพรรักษามะเร็งมั้ยครับ
17 ไม่เคี้ยวรึป่าวหรือว่าไข่มุกทำจากยางรถยนต์
18 ยังสรุปไม่ได้ว่าหนานเฉาเหว่ยรักษาโรคเบาหวา

In [10]:
print(len(nondup_x_test), len(nondup_x_tags_test), len(nondup_y_test))

4012 4012 4012


## Data saving

In [11]:
def to_jsonl(
    _path: str,
    _X: List[str],
    _Y: List[str],
    _X_TAGS: List[str] = None
) -> None:
    samples = []
    if _X_TAGS is not None:
        for x, x_tags, y in zip(_X, _X_TAGS, _Y):
            samples.append({"Text": x, "Token Tags": x_tags, "Document Tag": y})
    else:
        for x, y in zip(_X, _Y):
            samples.append({"Text": x, "Document Tag": y})
            
    f = open(_path, 'w')
    for sample in samples:
        f.write("%s\n" % json.dumps(sample, ensure_ascii=False))
    f.close()

In [12]:
to_jsonl(
    "train_v1.jsonl",
    x_train, y_train
)

to_jsonl(
    "val_v1.jsonl",
    x_val, x_val
)

to_jsonl(
    "test_v1.jsonl",
    nondup_x_test, nondup_y_test, nondup_x_tags_test
)